<a href="https://colab.research.google.com/github/v0001/data-analysis/blob/master/Analysis_hospitalBilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime 

%matplotlib inline

#  1. Import an event log (csv format only) as a pandas dataframe


In [ ]:
original = pd.read_csv('/content/drive/My Drive/★UNIST/2020 2학기/EI 1/data/hospital_billing.csv', index_col = 0)
original.head(2)

,case_id,activity,lifecycle,resource,timestamp,actorange,actred,blocked,casetype,closecode,diagnosis,flaga,flagb,flagc,flagd,iscancelled,isclosed,msgcode,msgcount,msgtype,speciality,state,version,activity_instance_id,.order
1,A,NEW,complete,ResA,2012-12-16 19:33:10,NaN,NaN,False,A,NaN,A,False,False,NaN,True,False,True,NaN,NaN,NaN,A,In progress,NaN,1,1
2,A,FIN,complete,NaN,2013-12-15 19:00:37,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,2,2


In [ ]:
original.columns

Index(['case_id', 'activity', 'lifecycle', 'resource', 'timestamp',
       'actorange', 'actred', 'blocked', 'casetype', 'closecode', 'diagnosis',
       'flaga', 'flagb', 'flagc', 'flagd', 'iscancelled', 'isclosed',
       'msgcode', 'msgcount', 'msgtype', 'speciality', 'state', 'version',
       'activity_instance_id', '.order'],
      dtype='object')

# 2. Rename the attributes as “caseid, activity, ts” if names are different (ts is for timestamp!)

In [ ]:
original = original.rename(columns = 
             {
              'case_id'             : 'caseid',
              'activity'            : 'activity',
              'timestamp'  : 'ts'
             })

In [ ]:
original.head(2)

,caseid,activity,lifecycle,resource,ts,actorange,actred,blocked,casetype,closecode,diagnosis,flaga,flagb,flagc,flagd,iscancelled,isclosed,msgcode,msgcount,msgtype,speciality,state,version,activity_instance_id,.order
1,A,NEW,complete,ResA,2012-12-16 19:33:10,NaN,NaN,False,A,NaN,A,False,False,NaN,True,False,True,NaN,NaN,NaN,A,In progress,NaN,1,1
2,A,FIN,complete,NaN,2013-12-15 19:00:37,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,2,2


# 3. Create an event log (= a new dataframe) retaining only the caseid, activity and ts attributes

In [ ]:
df_hospital_event = pd.DataFrame(data = original, columns= ['caseid', 'activity', 'ts'])
df_hospital_event.head()

,caseid,activity,ts
1,A,NEW,2012-12-16 19:33:10
2,A,FIN,2013-12-15 19:00:37
3,A,RELEASE,2013-12-16 03:53:38
4,A,CODE OK,2013-12-17 12:56:29
5,A,BILLED,2013-12-19 03:44:31


## -> remove a record with NA as a value of 'caseid'

In [ ]:
null_data = df_hospital_event[df_hospital_event['caseid'].isna()]
null_data

,caseid,activity,ts
213,NaN,NEW,2012-12-14 22:52:38


In [ ]:
df_hospital_event.dropna(subset = ['caseid'], axis = 0,  inplace = True)

In [ ]:
null_data = df_hospital_event[df_hospital_event['caseid'].isna()]
null_data

,caseid,activity,ts


# 4. Create a new dataframe with columns caseid, list of events in a case (ordered by timestamp)


In [ ]:
event_df = pd.DataFrame(columns = ['Caseid','List of events ordered by timestamp'])

In [ ]:
group_tmp = df_hospital_event.groupby('caseid')
id_list = df_hospital_event['caseid'].unique().tolist()

In [ ]:
for caseid in id_list:
    tmp_df = group_tmp.get_group(caseid).sort_values(by = 'ts', ascending=True)
    a = tmp_df['activity'].to_list()
    event_df=event_df.append({'Caseid':caseid, 'List of events ordered by timestamp':a},ignore_index=True) 

In [ ]:
event_df.head(3)

,Caseid,List of events ordered by timestamp
0,A,"[NEW, FIN, RELEASE, CODE OK, BILLED]"
1,B,"[NEW, DELETE]"
2,C,"[NEW, FIN, RELEASE, CODE OK, REOPEN, CHANGE DI..."


# 5. Augment the event log created at 4 with two new attributes: duration, event_number
duration = ts(last event in case) - ts(first event in case)
Event_number: number of events in a case


In [ ]:
event_df['Duration'] = np.nan
event_df['Event_number'] = np.nan
event_df.head(5)

,Caseid,List of events ordered by timestamp,Duration,Event_number
0,A,"[NEW, FIN, RELEASE, CODE OK, BILLED]",NaN,NaN
1,B,"[NEW, DELETE]",NaN,NaN
2,C,"[NEW, FIN, RELEASE, CODE OK, REOPEN, CHANGE DI...",NaN,NaN
3,D,"[NEW, FIN, RELEASE, CODE OK, BILLED]",NaN,NaN
4,E,"[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",NaN,NaN


In [ ]:
for caseid in id_list:
    tmp_df = group_tmp.get_group(caseid)
    time = pd.to_datetime(tmp_df['ts'])
    min_time= time.min()
    max_time = time.max()
    duration = max_time-min_time

    event_df.loc[event_df['Caseid']==caseid, ["Duration","Event_number"]] = duration, len(tmp_df)

In [ ]:
event_df.head(5)

,Caseid,List of events ordered by timestamp,Duration,Event_number
0,A,"[NEW, FIN, RELEASE, CODE OK, BILLED]",367 days 08:11:21,5.0
1,B,"[NEW, DELETE]",306 days 17:03:15,2.0
2,C,"[NEW, FIN, RELEASE, CODE OK, REOPEN, CHANGE DI...",129 days 10:27:51,10.0
3,D,"[NEW, FIN, RELEASE, CODE OK, BILLED]",389 days 00:50:51,5.0
4,E,"[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",123 days 00:42:42,6.0


In [ ]:
event_df.tail()

,Caseid,List of events ordered by timestamp,Duration,Event_number
16994,LTN,"[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...",237 days 18:54:24,6.0
16995,MTN,"[NEW, DELETE]",0 days 01:25:35,2.0
16996,NTN,"[NEW, FIN]",360 days 20:50:52,2.0
16997,OTN,"[NEW, FIN, RELEASE, CODE OK, BILLED]",369 days 21:54:42,5.0
16998,PTN,"[NEW, FIN, RELEASE, CODE OK, BILLED]",152 days 14:43:48,5.0


# 6. Find the longest/shortest case by duration - duration = ts(last event in case) - ts(first event in case)

In [ ]:
caseList_maxDuration = event_df['Caseid'][event_df['Duration']==event_df['Duration'].max()].to_list()
event_df['Duration'].max(),len(caseList_maxDuration), caseList_maxDuration

(Timedelta('1035 days 10:06:32'), 2, ['GTC', 'GTC'])

In [ ]:
event_df['Duration'].min()

Timedelta('0 days 00:00:00')

In [ ]:
caseList_minDuration = event_df['Caseid'][event_df['Duration']==event_df['Duration'].min()].to_list()
len(caseList_minDuration), caseList_minDuration

(3084,
 ['U',
  'V',
  'ZD',
  'AE',
  'CE',
  'DE',
  'EE',
  'FE',
  'GE',
  'HE',
  'IE',
  'JE',
  'KE',
  'LE',
  'ME',
  'NE',
  'OE',
  'PE',
  'QE',
  'RE',
  'SE',
  'TE',
  'UE',
  'VE',
  'WE',
  'XE',
  'TF',
  'IH',
  'LI',
  'MI',
  'OI',
  'QI',
  'RI',
  'SI',
  'TI',
  'IM',
  'JM',
  'KM',
  'LM',
  'MM',
  'NM',
  'OM',
  'PM',
  'QM',
  'RM',
  'SM',
  'TM',
  'UM',
  'VM',
  'TN',
  'UN',
  'VN',
  'WN',
  'XN',
  'ZO',
  'AP',
  'CP',
  'GQ',
  'HQ',
  'IQ',
  'JQ',
  'CR',
  'DR',
  'ER',
  'FR',
  'GR',
  'HR',
  'IR',
  'JR',
  'KR',
  'LR',
  'MR',
  'NR',
  'OR',
  'PR',
  'QR',
  'ZS',
  'TT',
  'UT',
  'VT',
  'WT',
  'XT',
  'YT',
  'ZT',
  'AU',
  'BU',
  'CU',
  'DU',
  'EU',
  'ZX',
  'AY',
  'BY',
  'CY',
  'DY',
  'EY',
  'FY',
  'GY',
  'HY',
  'IY',
  'JY',
  'KY',
  'LY',
  'MY',
  'NY',
  'PY',
  'QY',
  'RY',
  'FZ',
  'GZ',
  'HZ',
  'IZ',
  'JZ',
  'KZ',
  'LZ',
  'MZ',
  'NZ',
  'OZ',
  'PZ',
  'QZ',
  'SZ',
  'TZ',
  'UZ',
  'VZ',
  'WZ',
  '

In [ ]:
# Example of a Case with Min duration
event_df[event_df['Caseid']=='SQG']

,Caseid,List of events ordered by timestamp,Duration,Event_number
5191,SQG,[NEW],0 days 00:00:00,1.0
12191,SQG,[NEW],0 days 00:00:00,1.0


# 7. Fine the longest case(s) by number of events

In [ ]:
list_max_eventno= event_df['Caseid'][event_df['Event_number']==event_df['Event_number'].max()].to_list()
len(list_max_eventno)

1